<a href="https://colab.research.google.com/github/feliciakiani/PolitikPedia/blob/main/Machine%20Learning/ETL_Spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PySastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 2.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving spamDetection.h5 to spamDetection.h5


In [5]:
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def filters(text):
  filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
  regex_pattern = '[' + re.escape(filters) + ']'

  filtered_text = re.sub(regex_pattern, '', text)

  return filtered_text


factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
    return stemmer.stem(text)

def preprocessedText(text):
  text = filters(text)
  text = stemming(text)

  return text

In [ ]:
#gk perlu
with open('sentences_spam.txt', 'w', encoding='utf-8') as file:
    for sentences_spam in X:
        file.write(sentences_spam + '\n')

from google.colab import files

# Save the best model checkpoint to a file
files.download('sentences_spam.txt')

In [ ]:
!pip install mysql.connector
!pip install textblob
import mysql.connector
from textblob import TextBlob
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

from keras.models import load_model
model = load_model("spam_classification.h5")

#kodingannya taru dimana ? access modelnya taro dimana gitu

# Koneksi ke database Cloud SQL
conn = mysql.connector.connect(
    host='your-cloud-sql-host',
    user='your-username',
    password='your-password',
    database='your-database'
)

# Buat kursor
cursor = conn.cursor()

# Query data user
query_user = "SELECT id, username, email FROM users"
cursor.execute(query_user)
users = cursor.fetchall()

#-------------------------------------------------spam_Classification-------------------------------------------#
# Hyperparameter, jangan diganti
max_words = 100

# Loop through each user
for user_id, username, email in users:
    # Query data komentar for each user
    query_comments = f"SELECT id, comment_text FROM comments WHERE user_id = {user_id}"
    cursor.execute(query_comments)
    comments = cursor.fetchall()

    # Loop through each comment for the user
    for comment_id, comment_text in comments:

         # Apply preprocessing
        preprocessed_text = preprocessedText(comment_text)

        # Tokenize the text data
        Tokenizer = tf.keras.preprocessing.text.Tokenizer()
        tokenizer = Tokenizer(num_words=max_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True)
        # Read the content of 'sentences.txt' and split it into lines

        with open('sentences_spam.txt', 'r', encoding='utf-8') as file:
          tokenizer_sentences = file.read().splitlines()
        tokenizer.fit_on_texts(tokenizer_sentences)

        # Tokenize and pad the new text
        new_sequence = tokenizer.texts_to_sequences([preprocessed_text])
        new_padded_sequence = pad_sequences(new_sequence, maxlen=max_words, padding='post')

        # Make predictions
        prediction = model.predict(new_padded_sequence)

        # Extract the scalar value from the NumPy array
        confidence = prediction[0, 0]

        # Threshold for considering a label as positive
        threshold = 0.5

        # Interpret prediction
        predicted_class = 1 if confidence >= threshold else 0

        # Convert confidence to a format that can be handled by format method
        confidence_str = '{:.4f}'.format(confidence)

        print(f"User ID: {user_id}, Comment ID: {comment_id}, Predicted Class: {predicted_class} (Confidence: {confidence_str})")

ModuleNotFoundError: ignored